# Lab 3 - Module 3: Building a Perceptron

**Learning Objectives:**
- Understand the two-step perceptron process: weighted sum → activation
- Interactively adjust weights and bias to classify data
- See how the perceptron creates a decision boundary
- Connect perceptrons to the neural networks you'll learn about later

**Time:** ~20 minutes

---

**From Modules 1-2:** You learned that activation functions warp space and have different properties.

## Connection to Previous Modules

**Module 0:** Some patterns (XOR, circles) can't be separated by straight lines

**Module 1:** Activation functions warp space, creating curved boundaries from straight rules

**Module 2:** Different activations have different properties (range, smoothness, saturation)

**Today:** We'll build a complete **perceptron** by combining:
1. A weighted sum: `z = w₁×x₁ + w₂×x₂ + b`
2. An activation function: `output = activation(z)`

This is the fundamental building block of neural networks!

## 1. The Two-Step Process

A **perceptron** is just two simple steps:

### Step 1: Weighted Sum
Combine the inputs with **weights** and add a **bias**:

```
z = w₁×x₁ + w₂×x₂ + b
```

- **w₁, w₂** = weights (how much each input matters)
- **b** = bias (shifts the decision boundary)
- **z** = "pre-activation" (the value before applying activation)

This is just like the line equation from Lab 1! `z` measures "which side of the line" a point is on.

### Step 2: Activation
Apply an activation function to the weighted sum:

```
output = activation(z)
```

This introduces **nonlinearity** and often squashes the output to a useful range (like 0 to 1).

---

### The Complete Perceptron:
```
output = activation(w₁×x₁ + w₂×x₂ + b)
```

That's it! Two steps, one powerful model.

## 2. Setup: Create Multiple Datasets

Let's create several linearly separable datasets with different configurations. Each dataset has two distinct classes that can be separated by a straight line, but they require different perceptron parameters.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import FloatSlider, Dropdown, interact, VBox, HTML
from IPython.display import display

# Activation functions
def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

def tanh(z):
    return np.tanh(z)

def relu(z):
    return np.maximum(0, z)

def step(z):
    return (z > 0).astype(float)

# Create multiple datasets with different seeds
def create_dataset(dataset_num):
    """
    Generate linearly separable datasets with different configurations.
    
    Dataset 1: Diagonal separation (upper-right vs lower-left)
    Dataset 2: Vertical separation (left vs right)
    Dataset 3: Horizontal separation (top vs bottom)
    Dataset 4: Diagonal opposite (upper-left vs lower-right)
    Dataset 5: Tight clusters with small gap
    """
    np.random.seed(42 + dataset_num)
    n_points = 50
    
    if dataset_num == 1:
        # Diagonal: upper-right vs lower-left
        X_class0 = np.random.randn(n_points, 2) * 0.5 + np.array([-1.5, -1.5])
        X_class1 = np.random.randn(n_points, 2) * 0.5 + np.array([1.5, 1.5])
        name = "Diagonal (↗)"
    elif dataset_num == 2:
        # Vertical: left vs right
        X_class0 = np.random.randn(n_points, 2) * 0.6 + np.array([-1.8, 0])
        X_class1 = np.random.randn(n_points, 2) * 0.6 + np.array([1.8, 0])
        name = "Vertical (←→)"
    elif dataset_num == 3:
        # Horizontal: top vs bottom
        X_class0 = np.random.randn(n_points, 2) * 0.6 + np.array([0, -1.8])
        X_class1 = np.random.randn(n_points, 2) * 0.6 + np.array([0, 1.8])
        name = "Horizontal (↑↓)"
    elif dataset_num == 4:
        # Diagonal opposite: upper-left vs lower-right
        X_class0 = np.random.randn(n_points, 2) * 0.5 + np.array([-1.5, 1.5])
        X_class1 = np.random.randn(n_points, 2) * 0.5 + np.array([1.5, -1.5])
        name = "Diagonal (↖)"
    else:  # dataset_num == 5
        # Tight clusters
        X_class0 = np.random.randn(n_points, 2) * 0.4 + np.array([-1.0, -0.5])
        X_class1 = np.random.randn(n_points, 2) * 0.4 + np.array([1.2, 0.8])
        name = "Tight Clusters"
    
    X = np.vstack([X_class0, X_class1])
    y = np.hstack([np.zeros(n_points), np.ones(n_points)])
    
    return X, y, name

# Create all datasets
datasets = {}
for i in range(1, 6):
    X, y, name = create_dataset(i)
    datasets[i] = {'X': X, 'y': y, 'name': name}

print("✓ Five datasets created!")
print("\nEach dataset has:")
print("  • 50 blue points (Class 0)")
print("  • 50 red points (Class 1)")
print("  • Different orientations and separations")
print("\nYour goal: Find perceptron parameters that work for each dataset!")

## 3. Preview All Five Datasets

Let's see all five datasets side-by-side. Notice how they have different orientations and require different decision boundaries.

In [ ]:
# Preview all datasets
fig, axes = plt.subplots(2, 3, figsize=(16, 10), dpi=100)
axes = axes.flatten()

for i in range(1, 6):
    ax = axes[i-1]
    data = datasets[i]
    X, y, name = data['X'], data['y'], data['name']
    
    ax.scatter(X[y == 0, 0], X[y == 0, 1], 
              c='blue', s=60, alpha=0.6, label='Class 0', edgecolors='k', linewidths=1)
    ax.scatter(X[y == 1, 0], X[y == 1, 1], 
              c='red', s=60, alpha=0.6, label='Class 1', edgecolors='k', linewidths=1)
    
    ax.set_xlabel('x₁', fontsize=11)
    ax.set_ylabel('x₂', fontsize=11)
    ax.set_title(f'Dataset {i}: {name}', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)

# Hide the 6th subplot
axes[5].axis('off')

plt.tight_layout()
plt.show()

print("\nObservations:")
print("  • Dataset 1: Classes separated diagonally (↗)")
print("  • Dataset 2: Classes separated vertically (best separated by x₁)")
print("  • Dataset 3: Classes separated horizontally (best separated by x₂)")
print("  • Dataset 4: Classes separated diagonally (↖) - opposite of Dataset 1")
print("  • Dataset 5: Tighter clusters - requires more precise parameters")
print("\nAll are linearly separable - a perceptron can classify each one perfectly!")

## 4. Interactive Perceptron Builder

**Your task:** Choose a dataset, then adjust the weights (w₁, w₂), bias (b), and activation function to classify the data perfectly.

**How it works:**
1. For each point (x₁, x₂), compute: `z = w₁×x₁ + w₂×x₂ + b`
2. Apply activation: `output = activation(z)`
3. Classify: If output > threshold (0.5 for sigmoid/tanh, 0 for others), predict Class 1

The visualization shows:
- **Background color:** The perceptron's predictions across the entire space
- **Decision boundary:** The green line where the perceptron switches from Class 0 to Class 1
- **Data points:** Correctly classified points have solid borders, incorrect ones are marked with yellow X's

**Challenge:** Try to get 100% accuracy on all five datasets!

In [ ]:
def perceptron_classifier(dataset_num, w1, w2, b, activation_name):
    """
    Visualize perceptron decision boundary and classification.
    """
    # Get selected dataset
    data = datasets[dataset_num]
    X, y_true, name = data['X'], data['y'], data['name']
    
    # Select activation
    activations = {
        'Sigmoid': sigmoid,
        'Tanh': tanh,
        'ReLU': relu,
        'Step': step
    }
    activation = activations[activation_name]
    
    # Create grid for decision boundary visualization
    x1_grid = np.linspace(-4, 4, 200)
    x2_grid = np.linspace(-4, 4, 200)
    X1_grid, X2_grid = np.meshgrid(x1_grid, x2_grid)
    grid_points = np.column_stack([X1_grid.ravel(), X2_grid.ravel()])
    
    # Step 1: Weighted sum for all grid points
    z_grid = w1 * grid_points[:, 0] + w2 * grid_points[:, 1] + b
    
    # Step 2: Activation
    output_grid = activation(z_grid)
    
    # Classify based on threshold
    if activation_name in ['Sigmoid', 'Tanh']:
        threshold = 0.5 if activation_name == 'Sigmoid' else 0.0
    else:
        threshold = 0.5
    
    predictions_grid = (output_grid > threshold).astype(int).reshape(X1_grid.shape)
    
    # Classify data points
    z_data = w1 * X[:, 0] + w2 * X[:, 1] + b
    output_data = activation(z_data)
    predictions_data = (output_data > threshold).astype(int)
    
    # Compute accuracy
    accuracy = np.mean(predictions_data == y_true) * 100
    
    # Identify misclassified points
    correct = predictions_data == y_true
    
    # Create figure
    fig, ax = plt.subplots(figsize=(10, 10), dpi=100)
    
    # Plot decision regions
    ax.contourf(X1_grid, X2_grid, predictions_grid, 
               levels=[-0.5, 0.5, 1.5], colors=['lightblue', 'lightcoral'], alpha=0.3)
    
    # Plot decision boundary
    ax.contour(X1_grid, X2_grid, predictions_grid, 
              levels=[0.5], colors=['green'], linewidths=3, linestyles='solid')
    
    # Plot correctly classified points
    ax.scatter(X[correct & (y_true == 0), 0], X[correct & (y_true == 0), 1],
              c='blue', s=100, alpha=0.7, edgecolors='black', linewidths=1.5, label='Class 0 (correct)')
    ax.scatter(X[correct & (y_true == 1), 0], X[correct & (y_true == 1), 1],
              c='red', s=100, alpha=0.7, edgecolors='black', linewidths=1.5, label='Class 1 (correct)')
    
    # Plot misclassified points with thick red borders
    if not correct.all():
        ax.scatter(X[~correct, 0], X[~correct, 1],
                  c='yellow', s=150, alpha=0.9, edgecolors='red', linewidths=4, 
                  label='Misclassified', marker='X')
    
    ax.set_xlabel('x₁', fontsize=13)
    ax.set_ylabel('x₂', fontsize=13)
    ax.set_title(f'Dataset {dataset_num}: {name} | {activation_name} Activation\nAccuracy: {accuracy:.1f}%', 
                fontsize=14, fontweight='bold')
    ax.legend(fontsize=10, loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)
    
    plt.tight_layout()
    plt.show()
    
    # Feedback
    print(f"\nDataset {dataset_num}: {name}")
    print("="*70)
    print(f"  Weighted sum: z = {w1:.2f}×x₁ + {w2:.2f}×x₂ + {b:.2f}")
    print(f"  Activation: {activation_name}(z)")
    print(f"  Accuracy: {accuracy:.1f}% ({int(accuracy * len(y_true) / 100)}/{len(y_true)} correct)")
    
    if accuracy == 100:
        print("\n🎉 Perfect classification! Try another dataset to keep practicing!")
    elif accuracy >= 95:
        print("\n👍 Excellent! Just a tiny adjustment needed for perfection.")
    elif accuracy >= 85:
        print("\n😊 Good progress! Fine-tune the parameters to improve the boundary.")
    elif accuracy >= 70:
        print("\n💡 Getting there! Adjust weights to change the slope, bias to shift position.")
    else:
        print("\n💡 Tip:")
        print("   • For Dataset 2 (vertical), try large |w₁| and small w₂")
        print("   • For Dataset 3 (horizontal), try large |w₂| and small w₁")
        print("   • For diagonals, try similar |w₁| and |w₂| values")

# Interactive widget
print("Interactive Perceptron Builder")
print("="*70)
print("Select a dataset and adjust parameters to classify it perfectly.\n")

interact(
    perceptron_classifier,
    dataset_num=Dropdown(
        options=[
            (f'Dataset 1: {datasets[1]["name"]}', 1),
            (f'Dataset 2: {datasets[2]["name"]}', 2),
            (f'Dataset 3: {datasets[3]["name"]}', 3),
            (f'Dataset 4: {datasets[4]["name"]}', 4),
            (f'Dataset 5: {datasets[5]["name"]}', 5)
        ],
        value=1,
        description='Dataset:'
    ),
    w1=FloatSlider(min=-3, max=3, step=0.1, value=1.0, description='Weight w₁:', continuous_update=False),
    w2=FloatSlider(min=-3, max=3, step=0.1, value=1.0, description='Weight w₂:', continuous_update=False),
    b=FloatSlider(min=-5, max=5, step=0.1, value=0.0, description='Bias b:', continuous_update=False),
    activation_name=Dropdown(
        options=['Sigmoid', 'Tanh', 'ReLU', 'Step'],
        value='Sigmoid',
        description='Activation:'
    )
);

## 5. Understanding the Parameters

### Weights (w₁, w₂):
- Control the **orientation** (slope/angle) of the decision boundary
- Larger weight → that input dimension matters more
- If w₁ = w₂, the boundary has a 45° angle
- If w₁ ≫ w₂, the boundary is mostly vertical (x₁ matters more)
- If w₂ ≫ w₁, the boundary is mostly horizontal (x₂ matters more)

### Bias (b):
- Controls the **position** of the decision boundary
- Positive bias → shifts boundary toward negative direction
- Negative bias → shifts boundary toward positive direction
- Without bias (b=0), the boundary must pass through the origin

### Activation Function:
- For these linearly separable problems, **all activations should work equally well!**
- The activation determines the output range (0-1 for sigmoid, -1 to 1 for tanh, etc.)
- But the decision boundary location is determined by the weights and bias

### Tips for Each Dataset:
- **Dataset 1 (Diagonal ↗):** Need positive w₁ and w₂ (similar magnitudes)
- **Dataset 2 (Vertical):** Need large |w₁|, small w₂
- **Dataset 3 (Horizontal):** Need large |w₂|, small w₁
- **Dataset 4 (Diagonal ↖):** Need opposite signs for w₁ and w₂
- **Dataset 5 (Tight):** Requires precise adjustment of all three parameters

## 6. Connection to Neural Networks

You just built a **single perceptron** (one artificial neuron)!

A **neural network** is just:
- **Many perceptrons** arranged in layers
- Each perceptron has its own weights and bias
- Outputs from one layer become inputs to the next layer

### Single Perceptron (what you built):
```
Input layer     Perceptron                Output
    x₁  ────┐
            ├──→ [z = w₁x₁ + w₂x₂ + b] ──→ [σ(z)] ──→ output
    x₂  ────┘
```

### Multi-Layer Neural Network:
```
Input layer   Hidden layer      Output layer
    x₁  ────┐
            ├──→ [perceptron 1]──┐
    x₂  ────┤                    ├──→ [perceptron 4] ──→ output
            ├──→ [perceptron 2]──┤
            │                    │
            └──→ [perceptron 3]──┘
```

More layers and more perceptrons = more complex patterns can be learned!

**That's why neural networks are powerful:** They stack many simple perceptrons to create complex decision boundaries.

## Questions for Your Answer Sheet

**Q17.** Write down the two steps of a perceptron in your own words.

**Q18.** What do the weights (w₁, w₂) control about the decision boundary?

**Q19.** What does the bias (b) control about the decision boundary?

**Q20.** Compare Dataset 2 (vertical separation) and Dataset 3 (horizontal separation). For Dataset 2, which weight (w₁ or w₂) needed to be larger to get good classification? For Dataset 3, which weight needed to be larger? Explain why this makes sense.

**Q21.** For these linearly separable datasets, did the choice of activation function (Sigmoid vs Tanh vs ReLU vs Step) significantly change whether you could classify the data correctly? Why or why not?

**Q22.** In Section 6, we saw that neural networks are made of many perceptrons in layers. Why might having multiple layers of perceptrons allow neural networks to solve problems that a single perceptron cannot (like XOR or circles from Module 0)?

## Next Steps

1. **Answer Q17-Q22** on your answer sheet
2. **Return to the LMS** and continue to Module 4
3. In Module 4, you'll test the perceptron on harder problems (XOR and circles) to see its limits!